In [1]:
import os
import torch
import pandas as pd
import xarray as xr
import json
from dataset import ChallengeDataset
from torch.utils.data import DataLoader, IterableDataset
from torchinfo import summary
import torch.nn as nn
import torch.optim as optim
from ocf_blosc2 import Blosc2
import torch.nn.functional as F
from datetime import datetime, time, timedelta
import numpy as np


In [2]:
# selects the gpu if available (when running on Google Colab) otherwise on the local cpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

#### **Download data**

The next block downloads the data from huggingface. This is only required if working on Google Colab OR data has not yet been downloaded locally. Expect this cell to run for up to 30 minutes.

In [3]:
# # Download site locations (indices.json) and data (pv and satellite-hrv)

# if not os.path.exists("submission"):
#      os.makedirs("submission", exist_ok=True)
#      #Installing locally means you do not need to rerun this each time you restart the notebook
#      !curl -L https://raw.githubusercontent.com/climatehackai/getting-started-2023/main/indices.json --output indices.json

# if not os.path.exists("data"):
#     os.makedirs("data/pv/2020", exist_ok=True)
#     os.makedirs("data/pv/2021", exist_ok=True)
#     os.makedirs("data/satellite-hrv/2020", exist_ok=True)
#     os.makedirs("data/satellite-hrv/2021", exist_ok=True)

#     !curl -L https://huggingface.co/datasets/climatehackai/climatehackai-2023/resolve/main/pv/metadata.csv --output data/pv/metadata.csv

#      # Download data for June, July, August 2020 and 2021
#     for summer_months in range (6,9):
#           !curl -L https://huggingface.co/datasets/climatehackai/climatehackai-2023/resolve/main/pv/2020/{summer_months}.parquet --output data/pv/2020/{summer_months}.parquet
#           !curl -L https://huggingface.co/datasets/climatehackai/climatehackai-2023/resolve/main/pv/2021/{summer_months}.parquet --output data/pv/2021/{summer_months}.parquet     
#           !curl -L https://huggingface.co/datasets/climatehackai/climatehackai-2023/resolve/main/satellite-hrv/2020/{summer_months}.zarr.zip --output data/satellite-hrv/2020/{summer_months}.zarr.zip
#           !curl -L https://huggingface.co/datasets/climatehackai/climatehackai-2023/resolve/main/satellite-hrv/2021/{summer_months}.zarr.zip --output data/satellite-hrv/2021/{summer_months}.zarr.zip

##### **Load pv, hrv and indices**

In [3]:
# Load all pv data into a single dataframe
pv = []
for year in [2020, 2021]:
    for month in [6, 7, 8]:
        pv.append(pd.read_parquet(f"data/pv/{year}/{month}.parquet").drop("generation_wh", axis=1))  
pv = pd.concat(pv)
pv.index = pv.index.set_levels([pv.index.levels[0].tz_localize(None), pv.index.levels[1]])
 
# The parquet data here is similar to a dataframe. The "power" is the column with the other data types being indexes. The data is shaped with each timestamp being its own 
# subframe with the sites having their corresponding power (% of site capacity).  
hrv = []
for year in [2020, 2021]:
    for month in [6, 7, 8]:
        hrv.append(xr.open_dataset(f"data/satellite-hrv/{year}/{month}.zarr.zip", engine="zarr", chunks="auto"))

hrv = xr.concat(hrv, dim="time")

# Images are stored as vectors. The vectors are stored as an array of vectors. The arrays have a timestamp. Since there is only one channel (hrv)
# the array is a 1D set of vectors with the dimension being time. Read this to help you understand how this is being stored 
# https://tutorial.xarray.dev/fundamentals/01_datastructures.html
with open("indices.json") as f:
    site_locations = {
        data_source: {
            int(site): (int(location[0]), int(location[1]))
            for site, location in locations.items() #if site == '2607'#added this to run only 1 site location to understand how it works
        }
        for data_source, locations in json.load(f).items()
    }

#### **Create train, validation and test datasets**

In [26]:
horizon = 1 # forecast horizon in hours
crop_size = 1 # number of pixels of hrv image to crop around each site location
BATCH_SIZE = 32 # number of samples per batch

train_start_date = "2020-06-01"
train_end_date = "2020-07-01"
validation_start_date = "2020-07-08"
validation_end_date = "2020-07-15"

In [28]:
from dataset import ChallengeDataset

# train 
train_dataset = ChallengeDataset(pv, hrv, site_locations, start_date=train_start_date, end_date=train_end_date)
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, pin_memory = True)

# validation
validation_dataset = ChallengeDataset(pv, hrv, site_locations=site_locations, start_date=validation_start_date, end_date=validation_end_date) 
validation_dataloader = DataLoader(validation_dataset, batch_size=BATCH_SIZE, pin_memory=True)

# # test
# test_dataset = ChallengeDataset(pv, hrv, site_locations,
#                         start_date="2021-07-01", end_date="2020-08-31",
#                         crop_size = crop_size, horizon = horizon)
# test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, pin_memory = True)

# **Model Building**

In [29]:
layers = [4, 4, 4, 4] #For a deeper resnet with 16 total conv layers

def conv_block(in_channels, out_channels, kernel_size=1, stride=1, padding=0):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(inplace=True))

class BasicBlock(nn.Module):
    expansion = 1
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv_block(in_channels, out_channels, stride=stride)
        self.conv2 = conv_block(out_channels, out_channels)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x
        out = self.conv1(x)
        out = self.conv2(out)
        if self.downsample is not None:
            identity = self.downsample(x)
        out = out + identity
        return F.relu(out, inplace=False)

class ResNet(nn.Module):
    
    def __init__(self, block, layers):
        
        super(ResNet, self).__init__()
        self.in_channels = 12 #reduce the stride
        self.initial = nn.Identity()
        #self.maxpool = nn.MaxPool2d(kernel_size=1, stride=1, padding=0)
        self.layer1 = self._make_layer(block, 12, layers[0])
        self.layer2 = self._make_layer(block, 24, layers[1], stride=1)
        self.layer3 = self._make_layer(block, 48, layers[2], stride=1)
        self.layer4 = self._make_layer(block, 96, layers[3], stride=1)
        self.avgpool = nn.AdaptiveMaxPool2d((1, 1))
        # Adjust this linear layer based on the concatenated size of HRV and PV features
        self.fc = nn.Linear(96  + 12, 12)  

    def _make_layer(self, block, out_channels, num_blocks, stride=1):
        downsample = None
        if stride != 1 or self.in_channels != out_channels * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.in_channels, out_channels * block.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels * block.expansion),
            )
        layers = [block(self.in_channels, out_channels, stride, downsample)]
        self.in_channels = out_channels * block.expansion
        for _ in range(1, num_blocks):
            layers.append(block(self.in_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, pv, hrv ):
        #print("Initial HRV shape:", hrv.shape)  
        #print("Initial PV shape:", pv.shape) 
        #print(f"{pv[0]}")
        x = self.initial(hrv)
        #x = self.maxpool(x)
        #print("Shape after initial conv and maxpool:", x.shape)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        #print("Shape after ResNet_light blocks:", x.shape)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        pv = torch.flatten(pv, start_dim=1)


        if pv.dim() > 2:
            pv = torch.flatten(pv, start_dim=1)

        combined = torch.cat((x, pv), dim=1)

        if self.fc.in_features != combined.shape[1]:
            self.fc = nn.Linear(combined.shape[1], 12).to(combined.device)

        out = self.fc(combined)
        return out

# **Model Training**

In [19]:
def model_validation(model, criterion, validation_dataloader):
    model.eval() # This is used to set the model to evaluation mode
    with torch.no_grad(): # This is used to stop the model from storing gradients
        losses = []
        for time_ids, site_id, pv_features, hrv_features, pv_targets in validation_dataloader:
            pv_features, hrv_features, pv_targets = pv_features.to(device, dtype=torch.float), hrv_features.to(device, dtype=torch.float), pv_targets.to(device, dtype=torch.float)
            predictions = model(pv_features, hrv_features)
            loss = criterion(predictions, pv_targets)
            losses.append(loss.item())

    model.train() # This is used to set the model back to training mode
    
    return sum(losses) / len(losses)

In [20]:
model = ResNet(BasicBlock, layers).to(device)
criterion = nn.MSELoss()
optimiser = optim.Adam(model.parameters(), lr=1e-3)

In [21]:
# model 
EPOCHS = 1
training_losses = []
validation_losses = []
epoch_train_losses = []
epoch_validation_losses = []
for epoch in range(EPOCHS):
    model.train()

    running_loss = 0.0 ##sets the starting loss at zero
    count = 0 #is used to keep track of the number of batches passed through the training model
  
    for i, (time_ids, site_id, pv_features, hrv_features, pv_targets) in enumerate(train_dataloader): 
        
        optimiser.zero_grad()
        predictions = model(
            pv_features.to(device, dtype=torch.float),
            hrv_features.to(device, dtype=torch.float),
        )
        loss = criterion(predictions, pv_targets.to(device, dtype=torch.float))
        loss.backward()
        optimiser.step()

        running_loss += loss.item() * pv_targets.size(0)
        count += pv_targets.size(0)

        if i % 200 == 199:
            
            batch_loss = running_loss / count
            training_losses.append(batch_loss)

            print(f"Epoch {epoch + 1}, {i + 1}: {batch_loss}")
            
            # print(f"     Training Loss: {batch_loss}")

            # validation_loss = model_validation(model, criterion, validation_dataloader)
            # validation_losses.append(validation_loss)
            # print(f"     Validation Loss: {validation_loss}\n")
            
    
    epoch_train_loss = running_loss / count
    epoch_train_losses.append(epoch_train_loss)

    # epoch_validation_loss = model_validation(model, criterion, validation_dataloader)
    # epoch_validation_losses.append(epoch_validation_loss)

    # print(f"Epoch {epoch + 1}, Training Loss: {epoch_train_loss}")
    # print(f"Epoch {epoch + 1}, Validation Loss: {epoch_validation_loss}")

Epoch 1, 200: 0.131649367576465
     Training Loss: 0.131649367576465
Epoch 1, 400: 0.07859283935744316
     Training Loss: 0.07859283935744316
Epoch 1, 600: 0.0573826653463766
     Training Loss: 0.0573826653463766
Epoch 1, 800: 0.04693743366195122
     Training Loss: 0.04693743366195122
Epoch 1, 1000: 0.042656528956023974
     Training Loss: 0.042656528956023974
Epoch 1, 1200: 0.04388317736156751
     Training Loss: 0.04388317736156751
Epoch 1, 1400: 0.042152924912682334
     Training Loss: 0.042152924912682334
Epoch 1, 1600: 0.03998660125609604
     Training Loss: 0.03998660125609604
Epoch 1, 1800: 0.037808177487070986
     Training Loss: 0.037808177487070986
Epoch 1, 2000: 0.03673040943511296
     Training Loss: 0.03673040943511296
Epoch 1, 2200: 0.035283012432839975
     Training Loss: 0.035283012432839975
Epoch 1, 2400: 0.03315730240336658
     Training Loss: 0.03315730240336658
Epoch 1, 2600: 0.032297039485263385
     Training Loss: 0.032297039485263385
Epoch 1, 2800: 0.03125842

In [24]:
model_name = "ResNet MSE"
os.makedirs(f"models/experiments/MSEvMAE/{model_name}", exist_ok=True)

# Save the variables used to make the dataset to a text file
with open(f"models/experiments/MSEvMAE/{model_name}/data_summary.txt", "w") as f:
    f.write("BATCH_SIZE = "+ str(BATCH_SIZE)+ "\n"+ 
            "train_start_date = "+ train_start_date+ "\n"+
            "train_end_date = "+ train_end_date+ "\n"+
            "validation_start_date = "+ validation_start_date+ "\n"+
            "validation_end_date = "+ validation_end_date)
    
# Save the trained model for future predictions
torch.save(model.state_dict(), f"models/experiments/MSEvMAE/{model_name}/trained_model.pt")

# Create a DataFrame from the training_losses and validation_losses lists
df = pd.DataFrame({'Training Losses': training_losses})
# df = pd.DataFrame({'Training Losses': batch_losses, 'Validation Losses': validation_losses})

# Export the DataFrame to a CSV file
df.to_csv(f'models/experiments/MSEvMAE/{model_name}/losses.csv', index=False)

In [36]:
model_name = "ResNet MSE"
model = ResNet(BasicBlock, layers).to(device)
model.load_state_dict(torch.load(f"models/experiments/MSEvMAE/week 1/{model_name}/trained_model.pt"))
model.eval()

ResNet(
  (initial): Identity()
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Sequential(
        (0): Conv2d(12, 12, kernel_size=(1, 1), stride=(1, 1))
        (1): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
      (conv2): Sequential(
        (0): Conv2d(12, 12, kernel_size=(1, 1), stride=(1, 1))
        (1): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
    )
    (1): BasicBlock(
      (conv1): Sequential(
        (0): Conv2d(12, 12, kernel_size=(1, 1), stride=(1, 1))
        (1): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
      (conv2): Sequential(
        (0): Conv2d(12, 12, kernel_size=(1, 1), stride=(1, 1))
        (1): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
    

In [37]:
def _eval_visual(dataloader, model, device):
    model.eval()

    predictions_list = []
    timestamps_list = []
    pv_targets_list = []  # List to store pv_targets for each batch

    with torch.no_grad():
        for i, (time_ids, site_id, pv_features, hrv_features, pv_targets) in enumerate(dataloader):
            hrv_features = hrv_features.to(device, dtype=torch.float)
            pv_features = pv_features.to(device, dtype=torch.float)
            pv_targets = pv_targets.to(device, dtype=torch.float)
            
            batch_predictions = model(pv_features, hrv_features)
            batch_predictions = batch_predictions.cpu().numpy()
            batch_pv_targets = pv_targets.cpu().numpy()  # Convert pv_targets to numpy array

            # Timestamp processing as before
            if isinstance(time_ids[0], tuple) or isinstance(time_ids[0], list):
                single_timestamp = time_ids[0][0]
            else:
                single_timestamp = time_ids[0]
            if isinstance(single_timestamp, datetime):
                timestamp = single_timestamp.strftime('%Y-%m-%d %H:%M:%S')
            else:
                timestamp = str(single_timestamp)
            
            # Append each batch's data to the lists
            predictions_list.append(batch_predictions)
            pv_targets_list.append(batch_pv_targets)  # Append pv_targets to its list
            batch_timestamps = [timestamp] * batch_predictions.shape[0]
            timestamps_list.extend(batch_timestamps)

    # Concatenate all collected arrays into single numpy arrays
    predictions = np.concatenate(predictions_list, axis=0)
    pv_targets = np.concatenate(pv_targets_list, axis=0)  # Concatenate all pv_targets

    # Convert to DataFrame
    predictions_df = pd.DataFrame(predictions)
    pv_targets_df = pd.DataFrame(pv_targets, columns=[f'target_{i}' for i in range(pv_targets.shape[1])])
    timestamps_df = pd.DataFrame(timestamps_list, columns=['timestamp'])

    # Combine timestamps, predictions, and targets by using index alignment
    final_df = pd.concat([timestamps_df, predictions_df, pv_targets_df], axis=1)
    # final_df.to_csv('predictions.csv', index=False)

    return final_df

# Usage
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
prediction_df = _eval_visual(validation_dataloader, model, device)
prediction_df[['date', 'time']] = prediction_df['timestamp'].str.split('T', expand=True)
timestamp_index = prediction_df.columns.get_loc('timestamp')
prediction_df.insert(timestamp_index, 'date', prediction_df.pop('date'))
prediction_df.insert(timestamp_index + 1, 'time', prediction_df.pop('time'))
prediction_df.drop('timestamp', axis=1, inplace=True)

prediction_df.to_csv(f'models/experiments/MSEvMAE/week 2/{model_name}_predictions.csv', index=False)